<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MC_Antit%C3%A9ticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Monte Carlo y Variables antitéticas**

In [88]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time
from random import random

**Idea general:**

1. Generamos números aleatorios uniformes en el intervalo $$ [0,1] $$
2. Evaluamos la función $$ g(x) = \sqrt{\arctan(x)} $$ en esos puntos.  
3. Calculamos el promedio de esos valores.  

In [89]:
def g(x):
  return np.sqrt(np.arctan(x))

Por la ley de los grandes números, ese promedio se va acercando al valor real de la integral
a medida que aumentamos el número de simulaciones \(n\).

In [90]:
n = 5000
repeticiones=50

**Variables Antitéticas:**

Un problema con Monte Carlo es que, aunque es insesgado (en promedio da el valor correcto), puede tener mucha **varianza**.  

Para reducir esa variabilidad, usamos el truco de las variables antitéticas:

- Si tenemos un número aleatorio \(r\), también usamos su complementario \(1-r\).  
- Esto se hace porque \(r\) y \(1-r\) están correlacionados de forma negativa:  
  si un valor da un resultado alto, el otro tiende a compensar.  
- Así, al promediarlos, la estimación fluctúa menos.  

El nuevo estimador se define como:

$$ \hat{\theta}_{ant} = \frac{\hat{\theta}_1(r) + \hat{\theta}_1(1-r)}{2} $$

##**Método Crudo**

In [91]:
def crudo_antitetico(n):
    suma_r = 0
    suma_1r = 0
    for i in range(n):
        r = rd.random()
        suma_r += g(r)          # theta1(r)
        suma_1r += g(1-r)       # theta1(1-r)
    # Promedios individuales y combinado
    I1 = suma_r / n
    I2 = suma_1r / n
    I12 = (I1 + I2) / 2
    return I1, I2, I12

In [92]:
def evaluar_antitetico(n, repeticiones):
    inicio_tiempo = time.time()

    lista_I1 = []
    lista_I2 = []
    lista_I12 = []

    for _ in range(repeticiones):
        i1, i2, i12 = crudo_antitetico(n)
        lista_I1.append(i1)
        lista_I2.append(i2)
        lista_I12.append(i12)

    fin_tiempo = time.time()
    tiempo_total = fin_tiempo - inicio_tiempo

    # Medias y varianzas de cada caso
    mu_I1, var_I1 = np.mean(lista_I1), np.var(lista_I1)
    mu_I2, var_I2 = np.mean(lista_I2), np.var(lista_I2)
    mu_I12, var_I12 = np.mean(lista_I12), np.var(lista_I12)

    return tiempo_total, (mu_I1, var_I1), (mu_I2, var_I2), (mu_I12, var_I12)

##**Método Acierto y error**

In [93]:
def aciertoerror_antitetico(n):
    aciertos_r = 0
    aciertos_1r = 0
    for i in range(n):
        x = rd.random()
        y = rd.random()
        # Usamos r
        if y <= g(x):
            aciertos_r += 1
        # Usamos 1-r
        if y <= g(1-x):
            aciertos_1r += 1
    I1 = aciertos_r / n
    I2 = aciertos_1r / n
    I12 = (I1 + I2) / 2
    return I1, I2, I12

In [94]:
def evaluar_aciertoerror_antitetico(n, repeticiones):
    inicio_tiempo = time.time()

    lista_I1 = []
    lista_I2 = []
    lista_I12 = []

    for _ in range(repeticiones):
        i1, i2, i12 = aciertoerror_antitetico(n)
        lista_I1.append(i1)
        lista_I2.append(i2)
        lista_I12.append(i12)

    fin_tiempo = time.time()
    tiempo_total = fin_tiempo - inicio_tiempo

    mu_I1, var_I1 = np.mean(lista_I1), np.var(lista_I1)
    mu_I2, var_I2 = np.mean(lista_I2), np.var(lista_I2)
    mu_I12, var_I12 = np.mean(lista_I12), np.var(lista_I12)

    return tiempo_total, (mu_I1, var_I1), (mu_I2, var_I2), (mu_I12, var_I12)

In [95]:
t_crudo, C1, C2, C12 = evaluar_antitetico(n, repeticiones) # Evaluamos crudo

t_acierto, A1, A2, A12 = evaluar_aciertoerror_antitetico(n, repeticiones) # Evaluamos acierto-error

Al repetir muchas veces los experimentos obtenemos tres estimadores:

- **theta1(r):** usando solo \(r\).  
- **theta1(1-r):** usando solo \(1-r\).  
- **Antitético:** combinación de ambos.  

Los tres dan aproximadamente la misma media (cercana al valor real de la integral), pero el **antitético tiene menor varianza**.  

- La media estimada no cambia.  
- Pero el resultado es más **estable** entre repeticiones.  

In [96]:
print("   Método        Estimador       Media        Varianza")
print("---------------------------------------------------------------")
print(f"  Crudo         theta1(r)     {C1[0]:.6f}    {C1[1]:.6f}")
print(f"  Crudo         theta1(1-r)   {C2[0]:.6f}    {C2[1]:.6f}")
print(f"  Crudo         Antitético    {C12[0]:.6f}    {C12[1]:.6f}")
print("---------------------------------------------------------------")
print(f"  Acierto-Err   theta1(r)     {A1[0]:.6f}    {A1[1]:.6f}")
print(f"  Acierto-Err   theta1(1-r)   {A2[0]:.6f}    {A2[1]:.6f}")
print(f"  Acierto-Err   Antitético    {A12[0]:.6f}    {A12[1]:.6f}")
print("===============================================================")
print(f"Tiempo Crudo: {t_crudo:.4f} seg")
print(f"Tiempo Acierto-Error: {t_acierto:.4f} seg")

   Método        Estimador       Media        Varianza
---------------------------------------------------------------
  Crudo         theta1(r)     0.630288    0.000007
  Crudo         theta1(1-r)   0.629468    0.000006
  Crudo         Antitético    0.629878    0.000001
---------------------------------------------------------------
  Acierto-Err   theta1(r)     0.630364    0.000043
  Acierto-Err   theta1(1-r)   0.630896    0.000042
  Acierto-Err   Antitético    0.630630    0.000026
Tiempo Crudo: 1.0621 seg
Tiempo Acierto-Error: 1.0717 seg


El método de Monte Carlo nos permite aproximar integrales complicadas mediante simulaciones aleatorias.  

- La versión **cruda** es directa, pero ruidosa.  
- La versión de **acierto y error** es intuitiva (área bajo la curva), aunque menos eficiente.  
- El uso de **variables antitéticas** mejora la precisión reduciendo la varianza sin necesidad de aumentar \(n\).  

La media sigue siendo la misma, pero el estimador antitético es “más confiable” porque varía menos entre experimentos.